卷积神经网络（Convolutional Neural Network，CNN）

相邻层的所有神经元之间都有连接，这称为全连接（fully connected），在全连接的神经网络中，Affine 层后面跟着激活函数ReLu层或者Sigmoid层。
而CNN中新增了Convolution层和Pooling层。CNN的层的连接顺序是“Convolution - ReLU -（Pooling）”（Pooling层有时会被省略）。这可以理解为之前的“Afne - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。靠近输出的层中使用了之前的“Afne - ReLU”组合。此外，最后的输出层中使用了之前的“Afne - Softmax”组合。这些都是一般的CNN中比较常见的结构。

全连接层存在的问题：

    之前介绍的全连接的神经网络中使用了全连接层（Affine层）。在全连接层中，相邻的神经元全部链接在一起，输出的数量可以任意决定。

    但是，这种全连接的结构有一个问题。就是数据的形状被忽视了，图像数据是三维数据（高、长、深），而全连接层的输入是需要将3维数据拉平为1维数据。

    为了解决这个问题，卷积神经网络（Convolutional Neural Network，CNN）被提出了。

    图象是3维形状，这个形状中应该含有重要的空间信息。比如，空间上邻近的像素为相似的值、RBG的各个通道之间分别有密切的关联性、相距较远的像素之间没有什么关联等，3维形状中可能隐藏有值得提取的本质模式。但是，因为全连接层会忽视形状，将全部的输入数据作为相同的神经元（同一维度的神经元）处理，所以无法利用与形状相关的信息，而卷积层可以保持形状不变，当输入数据是图像时，卷积层会以3维数据的形式接收输入数据，并同样以3维数据的形式输出至下一层，因此，在CNN中，可以（有可能）正确理解图像等具有形状的数据。

    另外，CNN中，有时将卷积层的输入输出数据称为特征图（feature map）。其中，卷积层的输入数据称为输入特征图（input feature map），输出数据称为输出特征图（output feature map）。

    

#### 卷积运算

对于输入数据，卷积运算以一定间隔滑动卷积核，将卷积核中的元素和输入数据中的元素一一对应相乘，并将乘积相加，得到输出数据中的一个元素。这里，乘积和相加的运算称为元素wise乘法（elementwise multiplication）和加法（addition）。

在全连接的神经网络中，除了权重参数，还存在偏置。CNN中，滤波器的参数就对应之前的权重。并且，CNN中也存在偏置。偏置通常只有1个（1×1），这个值和卷积后的每个元素相加作为输出数据

#### 填充

在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding），是卷积运算中经常会用到的处理。使用填充主要是为了调整输出的大小。比如，对大小为(4,4)的输入数据应用(3,3)的滤波器时，输出大小变为(2,2)，相当于输出大小比输入大小缩小了2个元素。这在反复进行多次卷积运算的深度网络中会成为问题。为什么呢？因为如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为1，导致无法再应用卷积运算。为了避免出现这样的情况，就要使用填充。在刚才的例子中，将填充的幅度设为1，那么相对于输入大小(4,4)，输出大小也保持为原来的(4,4)。因此，卷积运算就可以在保持空间大小不变的情况下将数据传给下一层。


#### 步幅

应用卷积核的位置间隔称为步幅（stride）。增大步幅后，输出大小会变小。而增大填充后，输出大小会变大。
假设输入大小为(H,W)，滤波器大小为(FH,FW)，输出大小为(OH,OW)，填充为P，步幅为S。
输出大小的计算如下式所示。

$$OH=\frac{H+2P-FH}{S}+1$$
$$OW=\frac{W+2P-FW}{S}+1$$

当输出大小无法除尽时（结果是小数时），需要采取报错等对策。根据深度学习的框架的不同，当值无法除尽时，有时会向最接近的整数四舍五入，不进行报错而继续运行。


#### 3维数据的卷积运算

图像是3维数据，除了高、长方向之外，还需要处理通道方向，需要注意的是，在3维数据的卷积运算中，输入数据和卷积核的通道数要设为相同的值，卷积核大小可以自由设定，但是每个通道的卷积核大小要相同。

通道数只能设定为和输入数据的通道数相同的值 ！！！！！！

通道数为C、高度为H、长度为W的数据的形状可以写成（C,H,W）。滤波器也一样，要按（channel, height,width）的顺序书写。比如，通道数为C、滤波器高度为FH（Filter Height）、长度为FW（Filter Width）时，可以写成（C,FH,FW）。

（C,H,W） *  (C,FH,FW) -----> (1,OH,OW)

这样一个卷积核完成对一张图片的卷积运算后，生成的一个输出数据（即一张特征图）形状为（1，OH，OW）。所谓1张特征图，就是通道数为1 的特征图。


要生成多个特征图，需要设定多个卷积核。比如，设定为3个卷积核，就会生成3个特征图。FN个卷积核会生成FN个特征输出图将这FN个特征图汇集在一起，就得到了形状为(FN,OH,OW)的方块。将这个方块传给下一层，就是CNN的处理流。

（C,H,W） *  (FN,C,FH,FW) -----> (FN,OH,OW)


每个通道只有一个偏置，偏置的形状是(FN,1,1)，滤波器的输出结果的形状是(FN,OH,OW)对滤波器的输出结果(FN,OH,OW)按通道加上相同的偏置值

(C, H, W) * (FN, C, FH, FW) ------>  (FN, OH, OW)  +  (FN, 1, 1) ------> (FN, OH, OW)

#### 批处理

神经网络的处理中进行了将输入数据打包的批处理。卷积运算也同样进行批处理。因此，需要将在各层间传递的数据保存为4维数据。按(batch_num,channel,height,width)的顺序保存数据

(N, C, H, W) * (FN, C, FH, FW) ------>  (N,FN, OH, OW)  +  (FN, 1, 1) ------> (N, FN, OH, OW)
     |                                         |                                   |
     |                                         |                                   |
  N个数据                                     N个数据                             N个数据

#### 池化层

池化是缩小高、长方向上的空间运算。

一般来说，池化窗口大小会和步幅设置为相同的值。

两种池化方式： 1、MAX池化     2、AVERAGE池化

池化层的特征：

- 没有要学习的参数 ：池化层和卷积层不同，没有要学习的参数。池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。
- 通道数不发生改变 ：经过池化运算，输入数据和输出数据的通道数不会发生变化，计算是按照通道独立进行的。

#### 卷积层和池化层的实现

4维数组：CNN中各层间传递的数据是4维数据。所谓4维数据，比如数据的形状是(10,1,28,28)，则它对应10个高为28、长为28、通道为1的数据。

In [ ]:
import numpy as np
x=np.random.rand(10,1,28,28)
print(x.shape)
#print(x[0])
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)

print(x[0,0]) #或者x[0][0],访问第一个数据的第1个通道的空间数据


CNN中处理的是4维数据，因此卷积运算的实现看上去会很复杂，但是通过使用下面要介绍的im2col这个技巧，问题就会变得很简单。

如果老老实实地实现卷积运算，估计要重复好几层的for语句。这样的实现有点麻烦，而且，NumPy中存在使用for语句后处理变慢的缺点（NumPy 中，访问元素时最好不要用for语句）。这里，我们不使用for语句，而是使用im2col这个便利的函数进行简单的实现。

im2col这个名称是“image to column”的缩写，翻译过来就是“从图像到矩阵”的意思

im2col是一个函数，将输入数据展开以适合滤波器（权重）。

对3维的输入数据应用im2col后，数据转换为2维矩阵（正确地讲，是把包含批数量的4维数据转换成了2维数据）

对于输入数据，将应用滤波器的区域（3维方块）横向展开为1列。im2col会在所有应用滤波器的地方进行这个展开处理。

使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为1列，并计算2个矩阵的乘积即可，最后转换（reshape）为输出数据的大小

In [8]:
import sys,os
sys.path.append(os.pardir)
from common.util import im2col
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


这里举了两个例子。第一个是批大小为1、通道为3的7×7的数据，第二个的批大小为10，数据形状和第一个相同。分别对其应用im2col函数，在这两种情形下，第2维的元素个数均为75。这是滤波器（通道为3、大小为5×5）的元素个数的总和。批大小为1时，im2col的结果是(9,75)。而第2 个例子中批大小为10，所以保存了10倍的数据，即(90,75)。

In [ ]:
#使用im2col 来实现卷积层
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)        
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        col = im2col(x, FH, FW, self.stride, self.pad)        
        col_W = self.W.reshape(FN, -1).T # 滤波器的展开
        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return out

卷积层的初始化方法将滤波器（权重）、偏置、步幅、填充作为参数接收。滤波器是(FN, C, FH, FW)的4维形状。另外，FN、C、FH、FW分别是Filter Number（滤波器数量）、Channel、Filter Height、Filter Width的缩写。

用im2col展开输入数据，并用reshape将滤波器展开为2维数组。然后，计算展开后的矩阵的乘积。

展开滤波器的部分,将各个滤波器的方块纵向展开为1列。这里通过reshape(FN,-1)将参数指定为-1，这是reshape的一个便利的功能。通过在reshape时指定为-1，reshape函数会自动计算-1维度上的元素个数，以使多维数组的元素个数前后一致。比如，(10,3,5,5)形状的数组的元素个数共有750个，指定reshape(10,-1)后，就会转换成(10,75)形状的数组。

forward的实现中，最后会将输出大小转换为合适的形状。转换时使用了NumPy的transpose函数。transpose会更改多维数组的轴的顺序。通过指定从0开始的索引（编号）序列，就可以更改轴的顺序。

在进行卷积层的反向传播时，必须进行im2col的逆处理,卷积层的反向传播的实现在common/layer.py中

池化层的实现

池化的情况下，在通道方向是独立的，这一点和卷积不同。

1、首先在输入数据中，按照池化方式，在每个通道将池化区域的数据按行展开；
2、对展开的矩阵求各行的最大值，一列数据
3、将得到的这一列数据进行reshape，将数据转换为4维数组。

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 展开（1）
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 最大值（2）
        out = np.max(col, axis=1)

        # 转换（3）
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out
"""
1、展开输入数据
2、求各行的最大值
3、转换为合适的形状
"""

#### CNN的实现

网络的构成是“Convolution - ReLU - Pooling -Afne - ReLU - Afne - Softmax”，我们将它实现为名为SimpleConvNet的类。


In [11]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """简单的ConvNet

    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 输入大小（MNIST的情况下为784）
    hidden_size_list : 隐藏层的神经元数量的列表（e.g. [100, 100, 100]）
    output_size : 输出大小（MNIST的情况下为10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 指定权重的标准差（e.g. 0.01）
        指定'relu'或'he'的情况下设定“He的初始值”
        指定'sigmoid'或'xavier'的情况下设定“Xavier的初始值”
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """求损失函数
        参数x是输入数据、t是教师标签
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """求梯度（数值微分）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """求梯度（误差反向传播法）

        Parameters
        ----------
        x : 输入数据
        t : 教师标签

        Returns
        -------
        具有各层的梯度的字典变量
            grads['W1']、grads['W2']、...是各层的权重
            grads['b1']、grads['b2']、...是各层的偏置
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

In [ ]:
#使用这个SimpleConvNet学习MNIST数据集
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import matplotlib.pyplot as plt
from digit_recognition.mnist import load_mnist
from common.trainer import Trainer

# 读入数据
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 处理花费时间较长的情况下减少数据 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 保存参数
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 绘制图形
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


如果使用MNIST数据集训练SimpleConvNet，则训练数据的识别率为99.82%，测试数据的识别率为98.96%（每次学习的识别精度都会发生一些误差）。测试数据的识别率大约为99%，就小型网络来说，这是一个非常高的识别率。后面会通过进一步叠加层来加深网络，实现测试数据识别率超过99%。的网络

综上，卷积层和池化层的图像识别中必备的模块，CNN 可以有效读取图像中的某种特性，在手写数字识别中，还可以实现高精度的识别  

#### CNN的可视化

第1层权重的可视化

刚才我们对MNIST数据集进行了简单的CNN学习。当时，第1层的卷积层的权重的形状是(30,1,5,5)，即30个大小为5×5、通道为1的滤波器。滤波器大小是5×5、通道数是1，意味着滤波器可以可视化为1通道的灰度图像。现在，我们将卷积层（第1层）的滤波器显示为图像。这里，我们来比较一下学习前和学习后的权重源代码在ch07/visualize_filter.py中

学习前的滤波器是随机进行初始化的，所以在黑白的浓淡上没有规律可循，但学习后的滤波器变成了有规律的图像发现，通过学习，滤波器被更新成了有规律的滤波器，比如从白到黑渐变的滤波器、含有块状区域（称为blob）的滤波器等。

选择两个学习完的滤波器对输入图像进行卷积处理时的结果。我们发现“滤波器1”对垂直方向上的边缘有响应，“滤波器2”对水平方向上的边缘有响应。卷积层的滤波器会提取边缘或斑块等原始信息。而刚才实现的CNN会将这些原始信息传递给后面的层。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from simple_convnet import SimpleConvNet

def filter_show(filters, nx=8, margin=3, scale=10):
    """
    c.f. https://gist.github.com/aidiary/07d530d5e08011832b12#file-draw_weight-py
    """
    FN, C, FH, FW = filters.shape
    ny = int(np.ceil(FN / nx))

    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(FN):
        ax = fig.add_subplot(ny, nx, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i, 0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()


network = SimpleConvNet()
# 随机进行初始化后的权重
filter_show(network.params['W1'])

# 学习后的权重
network.load_params("params.pkl")
filter_show(network.params['W1'])

上面的结果是针对第1层的卷积层得出的。第1层的卷积层中提取了边缘或斑块等“低级”信息，那么在堆叠了多层的CNN中，各层中又会提取什么样的信息呢？根据深度学习的可视化相关的研究，，随着层次加深，提取的信息（正确地讲，是反映强烈的神经元）也越来越抽象。

如果堆叠了多层卷积层，则随着层次加深，提取的信息也愈加复杂、抽象，这是深度学习中很有意思的一个地方。最开始的层对简单的边缘有响应，接下来的层对纹理有响应，再后面的层对更加复杂的物体部件有响应。也就是说，随着层次加深，神经元从简单的形状向“高级”信息变化。换句话说，就像我们理解东西的“含义”一样，响应的对象在逐渐变化。

#### 具有代表性的CNN

LeNet
    
    是进行手写数字识别的网络,它有连续的卷积层和池化层（正确地讲，是只“抽选元素”的子采样层），最后经全连接层输出结果。和“现在的CNN”相比，LeNet有几个不同点。第一个不同点在于激活函数。LeNet中使用sigmoid函数，而现在的CNN中主要使用ReLU函数。此外，原始的LeNet中使用子采样（subsampling）缩小中间数据的大小，而现在的CNN中Max池化是主流。


AlexNet

    在LeNet问世20多年后，AlexNet被发布出来。AlexNet是引发深度学习热潮的导火线，不过它的网络结构和LeNet基本上没有什么不同,AlexNet叠有多个卷积层和池化层，最后经由全连接层输出结果。虽然结构上AlexNet和LeNet没有大的不同，但有以下几点差异。

    第一，AlexNet使用ReLU函数作为激活函数，而LeNet使用sigmoid函数。
    第二，AlexNet使用Dropout减少过拟合，而LeNet没有使用Dropout。
    第三，AlexNet使用重叠的最大池化（overlapping max pooling），而LeNet使用平均池化。
    第四，AlexNet在本地有多个GPU的情况下使用了cuDNN，这是一个针对GPU设计的卷积神经网络加速库。
    第五，AlexNet使用进行局部正规化的LRN（Local Response Normalization）层。

    关于网络结构，LeNet和AlexNet没有太大的不同。但是，围绕它们的环境和计算机技术有了很大的进步。具体地说，现在任何人都可以获得大量的数据。而且，擅长大规模并行计算的GPU得到普及，高速进行大量的运算已经成为可能。大数据和GPU已成为深度学习发展的巨大的原动力。
    

VGGNet
ResNet
